# Create Water Level Snapshot - pick time

**What this notebook does** This notebook creates a snapshot of latest water body conditions, appending the dam fill level as an attribute in a shapefile. This code needs to be run manually after new data has been ingested into DEA, and after the .txt files for each water bosy have been updated.

**Required inputs** 
- A shapefile to be appended with the latest data (`Shapefile_to_append`)
- A folder where all the `.txt files` are located from which the snapshot will be created (`Timeseries_dir`)
- The date for analysis needs to be updated below (`DateToExtractStr`)

**Date** December 2018

**Author** Claire Krause

In [1]:
import subprocess
import glob
import geopandas as gp
import re
import pandas as pd
from dateutil.relativedelta import relativedelta

## Set up the file paths, date to be extracted and output file names

`Timeseries_dir` is the folder where all the .txt files are located from which the snapshot will be created

`Shapefile_to_append` is the shapefile which the snapshot will be added to as an attribute

In [2]:
Timeseries_dir = '/g/data/r78/cek156/dea-notebooks/Dams/Dams2000to2018/Timeseries_jan20/'
Shapefile_to_append = '/g/data/r78/cek156/dea-notebooks/Dams/Dams2000to2018/AllNSW2000to201810LatestSnapshot.shp'

SnapshotShp = gp.read_file(Shapefile_to_append)

### Choose when you want the snapshot for

Note that there is a filter built in here that excludes observations that are more than a month from the chosen date.

In [3]:
DateToExtractStr = '20190120'
DateToExtract = pd.to_datetime(DateToExtractStr, format='%Y%m%d')

# Create filter range which only includes observations of +/- 45 days
DateFilterA = DateToExtract + relativedelta(days=-45)
DateFilterB = DateToExtract + relativedelta(days=45)

## Get a list of all of the files in the folder to loop through

In [4]:
TimeSeriesFiles = glob.glob(f'{Timeseries_dir}*.csv')

## Loop through and extract the relevant date from all the .txt files

In [5]:
for file in TimeSeriesFiles:
    # Get the ID
    NameComponents = re.split('\.|/', file)  # Splitting on '.' and '/'
    PolyID = NameComponents[-2]
    PolyID = int(PolyID)
    try:
        AllObs = pd.read_csv(file, parse_dates=[
                             'Observation Date'], index_col='Observation Date')
        x = AllObs.iloc[AllObs.index.get_loc(DateToExtract, method='nearest')]
        if(x.name > DateFilterA and x.name < DateFilterB):
            ObsDate = str(x.name)
            Pct = float(x['Wet pixel percentage'])
            FindPolyIndex = SnapshotShp.where(
                SnapshotShp['ID'] == PolyID).dropna(how='all').index.values[0]
            SnapshotShp.loc[(FindPolyIndex, f'{DateToExtractStr}')] = ObsDate
            SnapshotShp.loc[(FindPolyIndex, f'Pct{DateToExtractStr}')] = Pct
        else:
            print(x.name, 'is out of snapshot range.')
    except:
        print(f'Bad {PolyID}')

2018-12-05 23:43:37 is out of snapshot range.
2018-12-05 23:43:13 is out of snapshot range.
2018-11-19 23:42:51 is out of snapshot range.
2018-12-05 23:43:37 is out of snapshot range.
2018-11-18 23:47:30 is out of snapshot range.
2008-01-24 23:35:26 is out of snapshot range.
2018-11-19 23:43:15 is out of snapshot range.
2007-09-02 23:37:18 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-11-19 23:42:51 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2007-09-02 23:37:18 is out of snapshot range.
2018-11-28 23:36:39 is out of snapshot range.
2018-12-05 23:43:37 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2011-11-16 23:31:56 is out of snap

2011-10-15 23:32:32 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2008-02-09 23:35:11 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-19 23:43:15 is out of snapshot range.
2018-11-19 23:43:15 is out of snapshot range.
2018-11-19 23:42:51 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-12-05 23:42:49 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-11-19 23:42:51 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2018-12-05 23:44:00 is out of snapshot range.
2008-02-16 23:41:38 is out of snap

2018-11-19 23:44:03 is out of snapshot range.
2018-11-27 23:38:18 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-19 23:44:03 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2008-02-25 23:34:29 is out of snapshot range.
2018-12-05 23:44:24 is out of snapshot range.
2018-12-05 23:43:37 is out of snapshot range.
2018-11-18 23:47:06 is out of snapshot range.
2007-08-17 23:37:28 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-12-05 23:44:00 is out of snapshot range.
2008-01-24 23:35:26 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-12-05 23:43:37 is out of snapshot range.
2008-02-09 23:35:35 is out of snap

2018-11-19 23:42:03 is out of snapshot range.
2018-11-19 23:43:15 is out of snapshot range.
2018-11-27 23:39:06 is out of snapshot range.
2018-11-19 23:44:27 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-27 23:39:06 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-19 23:42:03 is out of snapshot range.
2018-11-27 23:39:06 is out of snapshot range.
2018-11-19 23:42:51 is out of snapshot range.
2018-11-11 23:40:17 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2018-11-19 23:44:03 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2018-10-18 23:43:34 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2018-10-02 23:42:40 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2008-02-25 23:34:29 is out of snapshot range.
2018-11-19 23:42:27 is out of snap

2018-11-27 23:38:42 is out of snapshot range.
2018-11-28 23:36:15 is out of snapshot range.
2018-11-18 23:47:06 is out of snapshot range.
2018-12-05 23:43:37 is out of snapshot range.
2018-11-27 23:39:06 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-11-19 23:42:03 is out of snapshot range.
2008-02-09 23:35:35 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2018-12-05 23:43:37 is out of snapshot range.
2018-11-28 23:36:15 is out of snapshot range.
2018-11-28 23:36:15 is out of snapshot range.
2008-02-09 23:35:11 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2018-11-19 23:42:51 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2008-01-24 23:35:26 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-10-26 23:39:27 is out of snap

2018-09-16 23:42:32 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2018-11-19 23:42:51 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2007-08-17 23:37:28 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2008-02-09 23:35:35 is out of snapshot range.
2008-02-25 23:34:29 is out of snapshot range.
2018-11-11 23:38:42 is out of snapshot range.
2018-12-05 23:44:00 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2008-03-12 23:34:11 is out of snapshot range.
2018-11-19 23:44:03 is out of snapshot range.
2008-01-24 23:35:26 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-12-05 23:43:37 is out of snapshot range.
2018-11-27 23:39:06 is out of snapshot range.
2018-11-27 23:39:06 is out of snapshot range.
2018-02-12 23:45:39 is out of snap

2018-11-28 23:36:15 is out of snapshot range.
2018-09-16 23:42:08 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2008-02-25 23:34:29 is out of snapshot range.
2008-01-15 23:42:12 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-12-05 23:42:49 is out of snapshot range.
2018-11-11 23:38:42 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-12-05 23:42:49 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-19 23:42:27 is out of snap

2018-11-27 23:38:42 is out of snapshot range.
2018-10-26 23:41:03 is out of snapshot range.
2018-11-19 23:42:51 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2018-12-05 23:42:49 is out of snapshot range.
2018-12-05 23:43:13 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-27 23:38:18 is out of snapshot range.
2018-11-19 23:43:39 is out of snapshot range.
2018-11-27 23:39:06 is out of snapshot range.
2018-12-05 23:44:48 is out of snapshot range.
2018-11-27 23:39:06 is out of snapshot range.
2018-11-19 23:42:51 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2018-11-27 23:39:06 is out of snapshot range.
2008-02-25 23:34:29 is out of snapshot range.
2018-12-05 23:44:48 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2008-02-09 23:35:11 is out of snapshot range.
2018-11-11 23:38:42 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2018-11-19 23:42:51 is out of snap

2008-01-24 23:35:26 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-10-18 23:43:34 is out of snapshot range.
2018-12-05 23:43:37 is out of snapshot range.
2018-12-05 23:43:37 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-11 23:40:41 is out of snapshot range.
2018-12-05 23:43:37 is out of snapshot range.
2008-02-09 23:35:35 is out of snapshot range.
2018-11-19 23:43:15 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-11-19 23:44:03 is out of snapshot range.
2018-11-27 23:38:42 is out of snapshot range.
2008-02-09 23:35:35 is out of snapshot range.
2018-12-05 23:42:25 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2018-11-19 23:42:27 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2011-11-16 23:31:56 is out of snap

## Write out the appended shapefile

In [8]:
Shapefile_to_append = '/g/data/r78/cek156/dea-notebooks/Dams/Dams2000to2018/AllNSW2000to201901LatestSnapshot.shp'

schema = gp.io.file.infer_schema(SnapshotShp)
schema['properties'][f'{DateToExtractStr}'] = 'str'
schema['properties'][f'Pct{DateToExtractStr}'] = 'float' 
SnapshotShp.to_file(Shapefile_to_append, schema = schema)

In [7]:
SnapshotShp

,area,ID,20181001,PctArea18,20171001,PctArea17,20161001,PctArea16,20181101,Pct2018110,20181122,Pct2018112,Nov-Oct,20181207,Pct2018120,geometry,20190120,Pct20190120
0,163125.0,0,2018-09-04 00:13:47,0.000000,2017-10-03 00:14:27.500000,0.000000,2016-09-14 00:14:26.500000,2.298851,2018-10-22 00:14:05,0.000000,2018-10-22 00:14:05,0.000000,0.000000,2018-11-23 00:14:09,0.000000,"POLYGON ((1140650 -3499450, 1140650 -3499425, ...",2019-01-10 00:14:04,0.0
1,116250.0,1,2018-09-04 00:13:47,0.000000,2017-10-03 00:14:27.500000,0.000000,2016-09-14 00:14:26.500000,9.677419,2018-10-22 00:14:05,0.000000,2018-11-15 00:10:43,0.000000,0.000000,2018-11-23 00:14:09,0.000000,"POLYGON ((1134275 -3498300, 1134225 -3498300, ...",2019-01-10 00:14:04,0.0
2,246875.0,2,2018-09-04 00:13:47,0.000000,2017-10-03 00:14:27.500000,0.000000,2016-10-07 00:20:40.500000,0.000000,2018-10-30 00:11:05,0.000000,2018-11-22 00:16:44,0.000000,0.000000,2018-11-30 00:20:19,0.000000,"POLYGON ((1102550 -3494675, 1102550 -3494650, ...",2019-01-17 00:20:13,0.0
3,140000.0,3,2018-09-04 00:13:47,0.000000,2017-10-03 00:14:27.500000,0.000000,2016-10-07 00:20:40.500000,0.000000,2018-10-22 00:14:05,0.000000,2018-11-22 00:16:44,0.000000,0.000000,2018-11-30 00:20:19,0.000000,"POLYGON ((1121800 -3493675, 1121800 -3493650, ...",2019-01-18 00:08:54,0.0
4,82500.0,4,2018-09-04 00:13:47,0.000000,2017-10-02 00:22:59.500000,0.000000,2016-10-07 00:20:40.500000,0.000000,2018-10-22 00:14:05,0.000000,2018-10-22 00:14:05,0.000000,0.000000,2018-11-30 00:20:19,0.000000,"POLYGON ((1121325 -3495775, 1121325 -3495750, ...",2019-01-17 00:20:13,0.0
5,188750.0,5,2018-09-04 00:13:47,0.000000,2017-10-03 00:14:27.500000,0.000000,2016-09-06 00:16:54.500000,0.000000,2018-10-22 00:14:05,0.000000,2018-10-22 00:14:05,0.000000,0.000000,2018-11-23 00:14:09,0.000000,"POLYGON ((1149050 -3490075, 1149050 -3490050, ...",2019-01-10 00:14:04,0.0
6,2149375.0,6,2018-09-04 00:13:47,0.000000,2017-10-03 00:14:27.500000,0.000000,None,NaN,2018-10-22 00:14:05,0.000000,2018-10-22 00:14:05,0.000000,0.000000,2018-11-23 00:14:09,0.000000,"POLYGON ((1133000 -3478050, 1133000 -3477975, ...",2019-01-10 00:14:04,0.0
7,366250.0,7,2018-09-04 00:13:47,0.000000,2017-10-03 00:14:27.500000,0.000000,2016-10-24 00:16:55.500000,0.000000,2018-10-22 00:14:05,0.000000,2018-11-15 00:10:43,0.000000,0.000000,2018-11-23 00:14:09,0.000000,"POLYGON ((1140275 -3472325, 1140250 -3472325, ...",2019-01-10 00:14:04,0.0
8,3125.0,8,2018-09-04 00:13:47,0.000000,2017-10-03 00:14:27.500000,0.000000,2016-09-14 00:14:26.500000,0.000000,2018-10-22 00:14:05,0.000000,2018-11-22 00:16:44,0.000000,0.000000,2018-11-23 00:14:09,0.000000,"POLYGON ((1120950 -3499375, 1120975 -3499375, ...",2019-01-17 00:20:13,0.0
9,4375.0,9,2018-09-04 00:13:47,0.000000,2017-10-03 00:14:27.500000,0.000000,2016-09-14 00:14:26.500000,100.000000,2018-10-22 00:14:05,0.000000,2018-11-15 00:10:43,0.000000,0.000000,2018-11-23 00:14:09,0.000000,"POLYGON ((1140775 -3498725, 1140800 -3498725, ...",2019-01-18 00:08:54,0.0
